In [ ]:
from IPython.display import YouTubeVideo, Markdown, SVG, Code
from functools import partial
YouTubeVideo_formato = partial(YouTubeVideo, modestbranding=1, disablekb=0,
                               width=640, height=360, autoplay=0, rel=0, showinfo=0)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from functools import partial


# Optimizando código Python

Como ya hemos mencionado **Python** es un lenguaje versátil pero poco eficiente en comparación a lenguajes de bajo nivel (C o Fortran)

Recordemos que **Python** es un lenguaje **interpretado**. Consideremos la operación

    z = x + y
    
Esta operación sencilla requiere de <font color="red">inferir el tipo</font> de $x$ e $y$ antes de sumarlos, luego debe <font color="red">escoger la función "suma" apropiada</font> y finalmente <font color="red">retornar el tipo correcto</font> de z

El costo de las operaciones en rojo se llama **overhead**, y es el **costo extra** de Python versus los lenguajes compilados

> Python nos da simpleza al costo de la eficiencia

Sin embargo, existen varias maneras de mejorar la eficiencia de un código escrito en Python. En esta clase veremos tres:

La primera y más fundamental es 

> **Conocer el lenguaje**: Usar la sintaxis y estructuras de Python adecuadamente 

Si tenemos un problema basado en arreglos podemos mejorar la eficiencia en varios órdenes de magnitud usando

> **Vectorización:** Cómputo basado en arreglos con `NumPy`

Finalmente, si las estructuras disponibles no son suficientes o si nuestros cálculos no son vectorizables se puede

> **Conectar con lenguajes de bajo nivel:** Uniendo Python y C con `Cython`

En la clase siguiente veremos como aprovechar las arquitecturas de CPU multi-nucleo con `Multiprocessing`

## Conocer el lenguaje para ganar eficiencia

**Python** tiene una curva de aprendizaje suave en comparación a lenguajes de más bajo nivel, es decir que sabiendo muy poco de **Python** ya somos capaces de escribir toda clase de rutinas

Esto tiene un efecto secundario negativo en algunas personas y especialmente en aquellos que ya saben otros lenguajes 

> **Grave error:** Usar Python como si fuera C (o otro lenguaje)

Python ofrece una gran cantidad de [funciones](https://docs.python.org/3/library/functions.html) y [módulos en su librería estándar](https://docs.python.org/3/library/index.html) que son sumamente eficientes. Usar la sintáxis y las [estructuras de datos](https://docs.python.org/3/tutorial/datastructures.html) de Python adecuadamente es el primer paso para escribir código eficiente 

> Tenga siempre presente lo aprendido en su curso de algoritmos y busque en la documentación de Python las estructuras de datos más apropiadas para cada problema

Si necesita repasar sobre algoritmos se recomienda el siguiente material

- Bibliografía complementaria del curso: [Effective Python](https://effectivepython.com/)
- [Tratado de algoritmos y estructuras de datos en Python](https://runestone.academy/runestone/books/published/pythonds/index.html)
- [Consejos de velocidad en la Python wiki](https://wiki.python.org/moin/PythonSpeed)
- [Complejidad temporal de distintas estructuras de Python](https://wiki.python.org/moin/TimeComplexity)

A continuación les dejo algunos consejos generales enfocados a Python

###  Evita usar `for` siempre que se pueda en favor de las funciones nativas

Muchas veces podemos evitar usar `for` con la estructura de datos o función adecuada

Para ejemplificar digamos que queremos sumar los valores absolutos de los elementos de una lista

In [ ]:
x = [x for x in range(100000)]

# Suma estilo C 
def suma_abs(data):
    resultado = 0
    for i in range(len(data)):
        if data[i] > 0:
            resultado += data[i]
        else:
            resultado -= data[i]
    return resultado

%timeit -r5 -n3 suma_abs(x)
suma_abs(x)

In [ ]:
# Mejora 1: No es necesario usar un índice, podemos iterar directamente en los elementos
def suma_abs(data):
    resultado = 0
    for element in data:
        if element > 0:
            resultado += element
        else:
            resultado -= element
    return resultado

%timeit -r5 -n3 suma_abs(x)
suma_abs(x)

In [ ]:
# Mejora 2: Operar como una comprensión de lista y luego usar la función sum de Python
%timeit -r5 -n3 sum([x if x> 0 else -x for x in x])
sum([x if x> 0 else -x for x in x])

In [ ]:
# Mejora 3: Usar las funciones sum, map y abs de Python
%timeit -r5 -n3 sum(list(map(abs, x)))
sum(list(map(abs, x)))

### No reinventes la rueda con las estructuras de datos

Verifica si la estructura que necesitas está implementada en Python antes de programarla tu

Como ejemplo digamos que queremos contar la cantidad de elementos de cada tipo en una lista. Podríamos escribir un contador como

In [ ]:
x2 = list(np.random.randint(10, size=10000))

# Un contador de elementos
def miCounter(data):
    count = {}
    for element in data:
        if element not in count:
            count[element] = 1
        else:
            count[element] +=1
    return count

%timeit -r7 -n1 miCounter(x2)
miCounter(x2)

Sin embargo, la clase contador ya existe en `collections`, y es mucho más eficiente que la implementación "a mano"

In [ ]:
from collections import Counter
%timeit -r7 -n1 Counter(x2)
Counter(x2)

### Ten atención con el overhead en funciones

Python tiene un overhead considerable cada vez que se llama una función

Se puede ganar desempeño haciendo *inlining* de funciones

In [ ]:
def cuadradomasuno(element):
    return element*element + 1

%timeit -r7 -n3 [cuadradomasuno(xi) for xi in x]
#Inlining: escribo la función textualmente en lugar de evaluarla
%timeit -r7 -n3 [xi*xi + 1 for xi in x] 
np.allclose([cuadradomasuno(xi) for xi in x], [xi*xi + 1 for xi in x] )

### Usa variables locales dentro de los loops

Si estamos obligados a usar `for` podemos ganar algo de rendimiento haciendo copias locales de atributos y funciones

Por ejemplo, digamos que queremos crear una lista con todos los elementos de otra lista que cumplen la condición

$$
\sin(x[i]) > 0 
$$


In [ ]:
import math

# iterando sobre la lista
def sin_pos(data):
    resultado = []
    for element in data:
        if math.sin(element) > 0:
            resultado.append(element)
    return resultado

%timeit -r5 -n3 sin_pos(x)
resultado1 = sin_pos(x)

Así se vería el código si hacemos variables locales para el método `append` y la función `sin`

In [ ]:
# Mejora: variables locales
def sin_pos(data):
    resultado = []
    append = resultado.append
    sin = math.sin
    for element in data:
        if sin(element) > 0:
            append(element)
    return resultado

%timeit -r5 -n3 sin_pos(x)
resultado2 = sin_pos(x)
display(np.allclose(resultado1, resultado2))

## Vectorización: Cómputo basado en arreglos con NumPy

Consideremos el escenario en que tenemos un arreglo de datos de gran tamaño y queremos hacer una operación sobre cada elemento

**Ejemplo:** Dado $\{x\}_i$ queremos encontrar

$$
y_i = \frac{1}{1 + e^{-x_i}}, \quad i = 1,2,\ldots, N
$$

Los cómputos de este tipo se catalogan como *SIMD*: Single Instruction Multiple Data, es decir que estamos haciendo una misma operación para todos los datos

Este problema se puede resolver usando un ciclo `for` 

```python    
    y = []
    for xi in x:
        y.append(1/(1+math.exp(-xi))
```

Sin embargo, ya sabemos que esto es ineficiente en "Python puro"

No olvidemos que las librerías de cómputo científico vistas en este curso nos ofrecen una mejor alternativa. En particular, **NumPy** nos provee de una estructura de arreglo multidimensional (ndarray) y funciones para operarla que están escritas en C y Fortran. Otras librerías como **Scipy** y **Pandas** se basan en **NumPy**

> Cuando utilizamos estas librerías estamos usando código compilado en lugar de interpretado 

Usando **NumPy** podemos reemplazar un ciclo `for` en problemas *SIMD* por operaciones que trabajan sobre todo el arreglo, estas sa llaman **operaciones vectoriales**

Por ejemplo el código que vimos anteriormente quedaría como (asumiendo que x es una lista)

```python
    x = np.array(x)
    y = 1/(1+np.exp(-x))
```
    
NumPy aplica la función exponencial a todo el arreglo x, luego aplica la aritmética (suma y división) a cada elemento del arreglo (broadcasting) y finalmente retorna un nuevo arreglo con el resultado. En general siempre que exista una operación de tipo *SIMD* podemos mejorar su rendimiento usando 

> **Vectorización:** Reemplazar un bucle/ciclo por operaciones vectoriales de **NumPy**

A continuación revisaremos mediante ejemplos como implementar este y otros conceptos para mejorar el rendimiento

### Reemplazar ciclo `for` por operaciones vectoriales

Las operaciones vectoriales son <font color="red"> funciones de NumPy</font> de tipo *element-wise* <font color="red">aplicadas sobre un ndarray</font>

Las funciones *element-wise* son aquellas que actuan sobre todos los elementos del arreglo de forma independiente

- En la clase de NumPy (unidad 1) revisamos algunos ejemplos: exponenciación, raíces, trigonometrícas, etc
- Las operaciones aritméticas entre ndarrays son por defecto *element-wise*

Luego si tenemos un problema SIMD escrito con un `for` sobre un conjunto de datos podemos 

1. Convertir los datos a ndarray
1. Escribir la operación con funciones de NumPy *element-wise*

y ganar considerablemente en eficiencia de forma directa. Tenga presente que cuando las operaciones de NumPy se ejecutan sobre un ndarray se está usando código compilado

Por ejemplo notemos la diferencia en tiempo de cómputo al hacer aritmética simple

In [ ]:
x_ndarray = np.random.randn(100000)
x_list = list(x_ndarray)

def operacion_simple(data):
    resultado = []
    append = resultado.append
    for elemento in data:
        append(elemento*elemento + elemento)
    return resultado

# Operación usando "for con mejoras"
%timeit -n3 -r7 operacion_simple(x_list)
# Operación usando numpy sobre un ndarray
%timeit -n3 -r7 x_ndarray*x_ndarray + x_ndarray
# Comparación entre los resultados
np.allclose(operacion_simple(x_list), x_ndarray*x_ndarray + x_ndarray)

Notemos que las funciones de NumPy son lentas cuando operan sobre tipos que no son ndarray

Para el ejemplo de $y_i = (1 + e^{-x_i})^{-1}, \quad i = 1,2,\ldots, N$

In [ ]:
from math import exp
# usando list comprehension (similar a un "for mejorado")
%timeit -n3 -r7 [1./(1.+exp(xi)) for xi in x_list]
# usando numpy sobre una lista
%timeit -n3 -r7 1./(1+np.exp(x_list))
# usando numpy sobre un ndarray
%timeit -n3 -r7 1./(1+np.exp(x_ndarray))
# Comparación entre los resultados
np.allclose(1./(1+np.exp(x_ndarray)), [1./(1.+exp(xi)) for xi in x_list])

Las operación sobre ndarray es casi un orden de magnitud más rápida

**¿Cómo se explica esto?**

Recuerde que una lista puede tener distintos tipos y estar guardada en distintos sectores de memoria. En cambio, el ndarray

- Tiene un tipo definido 
- Está guardado en bloques de memoria contiguos

Por ende el ndarray tiene un overhead de interpretador mucho menor

Además NumPy está escrito en C/Fortran, hacer un loop en memoría contigua en C es muy eficiente

Finalmente notar que NumPy puede compilarse con librerías de alto desempeño (openblas, MKL) aprovechando mejor las capacidades del hardware (Cache de CPU e instrucciones vectoriales de CPU)


### Convertir operaciones lógicas sobre arreglos en máscaras

Las operaciones lógicas en NumPy también son *element-wise* (Operaciones booleanas, clase NumPy, unidad 1)

Si queremos recuperar los elementos de un arreglo que cumplan una cierta condición podemos

1. Convertir los datos a ndarray
1. Escribir la operación como una máscara booleana de índices

Para el ejemplo anterior de recuperamos los elementos tal que $\sin(x_i)>0$

In [ ]:
import math

def sin_pos(data):
    resultado = []
    append = resultado.append
    sin = math.sin
    for element in data:
        if sin(element) > 0:
            append(element)
    return resultado

%timeit -r5 -n3 sin_pos(x_list)

%timeit -r5 -n3 x_ndarray[np.sin(x_ndarray) > 0.]

display(np.allclose(sin_pos(x_list), x_ndarray[np.sin(x_ndarray) > 0]))

### Usa operaciónes *in-place* y vistas de arreglos para evitar copia extra de datos

Cuando en NumPy hacemos

```python
    x = x*x
```

Se crea una copia interna de x*x, y luego x es direccionado a esa nueva copia. La zona de memoria con el valor original es luego eliminada por el *garbage-collector* de Python

Siempre que no necesitemos el valor original podemos usar operaciones *in-place* y ganar rendimiento, ya que evitamos la copia y eliminación adicional

In [ ]:
# Copia interna y cambio de referencia de x_ndarray 
%timeit -r10 -n10 x_ndarray = np.zeros(shape=(1000000)); y = x_ndarray*x_ndarray
# Sin copia interna
%timeit -r10 -n10 x_ndarray = np.zeros(shape=(1000000)); x_ndarray *= x_ndarray
# El resultado es idéntico
x_ndarray = np.zeros(shape=(1000000))
y = x_ndarray*x_ndarray
x_ndarray *= x_ndarray
np.allclose(x_ndarray, y)

Sea $x$ un ndarray, la operación

```python
    x[2:10] 
```

es una "vista de x". Recordar que las "vista de arreglo" no hacen copias en memoria ya que apuntan directamente al arreglo original. Es decir que si modificamos una vista modificamos el original

### Aprovecha el *broadcasting* automático de NumPy

Se pueden hacer operaciones vectorizadas con NumPy entre arreglos con tamaños distintos. En ese caso se aplican las reglas de *broadcasting* que vimos en la unidad 1 (clase NumPy). El *broadcasting* automático no hace copias en memoria

Ejemplo 1: Si le sumas una constante a un arreglo 1D, la constante se expande y se suma a cada elemento

In [ ]:
N = 1000000
x = np.zeros(shape=(N, ))
# broadcasting automático
%timeit -n10 -r10 x + 1.
# Agrandando y luego sumando
%timeit -n10 -r10 x + np.tile([1], len(x))
# mismo resultado
np.allclose(x + 1, x + np.tile([1], len(x)))

Ejemplo 2: Si le sumas un arreglo 1D a un arreglo 2D, el arreglo 1D se expande en la dimensión que le falta

In [ ]:
N, M = 10000, 1000
x = np.zeros(shape=(N, M)) # arreglo de NxM
y = np.zeros(shape=(N, )) # arreglo sin dimensión
y_ = y[:, np.newaxis] # arreglo de Nx1
# broadcasting automático
%timeit -n10 -r10 x + y_
display((x + y_).shape)
# Agrandando y luego sumando
%timeit -n10 -r10 x + np.tile(y_, (1, x.shape[-1]))
# mismo resultado
np.allclose(x + y_, x + np.tile(y_, (1, x.shape[-1])))

Ejemplo 3: Si sumas un arreglo 1D fila y un arreglo 1D columna se crea un arreglo 2D

In [ ]:
N, M = 10000, 1000
x = np.zeros(shape=(N, 1)) # arreglo columna de Nx1
y = np.zeros(shape=(1, M)) # arreglo fila de 1xM
# broadcasting automático
%timeit -n10 -r10 x + y
display((x+y).shape)
# Agrandando y luego sumando
%timeit -n10 -r10 np.tile(y, (x.shape[0], 1)) + np.tile(x, (1, y.shape[-1]))
np.allclose(x + y, np.tile(y, (x.shape[0], 1)) + np.tile(x, (1, y.shape[-1])))

Regla de oro

> Las dimensiones de dos arreglos son compatibles con *broadcast* automático si **son del mismo tamaño** o **una de ellas es igual a uno**

### Utiliza el ordenamiento en memoría más adecuado en cada caso

Como vimos en la unidad 1 (clase NumPy), los ndarray multidimensionales pueden guardarse en memoria como *row-major* (filas contiguas) o *column-major* (columnas contiguas)

Por defecto las matrices en NumPy son *row-major* pero podemos forzar la contigüidad usando el atributo `order` o trasponiendo (ojo que trasponer crea una copia)

Se puede verificar esto con el atributo `flag` de los ndarray

In [ ]:
data = np.arange(6).reshape(2, 3)
display(data)
# Verificamos los flags
display(data.flags)
# Así se ve row-major en memoria 
display(data.ravel())
# Verificamos los flags
dataT = data.T
display(dataT.flags)
# Así se ve column-major en memoria
display(dataT.ravel())

La mayoría de las funciones de NumPy funcionan más rápido en formato *row-major* (formato C). Pero algunas funciones de scipy (heredadas de Fortran) funcionan más rápido en formato *col-major* (formato Fortran)

Es recomendable verificar el orden en memoria que espera la función que vas a utilizar

In [ ]:
data = np.random.randn(10000, 10000) # (row-major)
# Sumando una fila
%timeit -n100 -r10 np.sum(data[0, :])
# Sumando todas las filas
%timeit -n10 -r10 np.sum(data, axis=1)

In [ ]:
# Sumando una columna
%timeit -n100 -r10 np.sum(data[:, 0])
# Sumando todas las columnas
%timeit -n10 -r10 np.sum(data, axis=0)

In [ ]:
# Sumando todas las columnas de la matriz traspuesta (column major)
%timeit -n10 -r10 np.sum(data.T, axis=0)

## Ejemplo formativo: Vectorizando el cálculo del "Set de Julia"

Consideremos el código en "Python puro" que vimos en la clase de *profiling*

In [ ]:
def evaluate(zi, zr, maxiters=50, cr=-0.835, ci=-0.2321):
    nit = 0
    zi2 = zi*zi
    zr2 = zr*zr
    while zi2 + zr2 <= 4. and nit < maxiters:
        zi = 2*zr*zi + ci
        zr = zr2 - zi2 + cr
        zr2 = zr*zr
        zi2 = zi*zi
        nit +=1
    return nit
    
def make_fractal(N, maxiters=50):
    image = []
    for i in range(N):
        row = []
        for j in range(2*N):
            zi = -1.0 + i*2/N
            zr = -2.0 + j*2/N
            row.append(evaluate(zi, zr, maxiters))
        image.append(row)
    return image

plt.figure(figsize=(8, 4), tight_layout=True)
plt.gca().axis('off')
plt.imshow(make_fractal(500));

Estudiando el código nos damos cuenta que la función `evaluate` se ejecuta de forma secuencial para cada uno de los 500x1000 píxeles

Si guardamos la imagen y los valores de $z$ como un `ndarray` podemos actualizar todos los pixeles "al mismo tiempo"

In [ ]:
def make_fractal_vectorized(N, maxiters=50):
    image = np.zeros(shape=(N, 2*N), dtype=np.int64)
    zi = np.linspace(-1, 1, num=N, endpoint=False).reshape(-1, 1)
    zr = np.linspace(-2, 2, num=2*N, endpoint=False).reshape(1, -1)
    # Creamos arreglos de 2NxN para zi y zr 
    zi = np.repeat(zi, repeats=2*N, axis=1)
    zr = np.repeat(zr, repeats=N, axis=0)
    cr, ci = -0.835, -0.2321
    nit = 0
    while nit < maxiters:
        zr2 = zr*zr # operaciones vectoriales
        zi2 = zi*zi
        mask = zr2 + zi2 <= 4. #mascara booleana
        image[mask] += 1
        zi[mask] = 2*zr[mask]*zi[mask] + ci
        zr[mask] = zr2[mask] - zi2[mask] +  cr
        nit += 1
    return image

# Los resultados comparados:
np.allclose(make_fractal(500), make_fractal_vectorized(500))

El resultado es idéntico al código secuencial en "Python puro". El tiempo total para calcular el fractal usando el código vectorizado es:

In [ ]:
%timeit -r3 -n1 make_fractal_vectorized(500)

Por otro lado el tiempo total para la rutina original es

In [ ]:
%timeit -r3 -n1 make_fractal(500)

El ***speed-up*** es el tiempo de la nueva rutina dividido el tiempo de referencia (rutina secuencial)

¿Cuánto es el *speed-up* en este caso?

**Mejoras propuestas para la rutina vectorizada**

- Usar operaciones *inplace* para aumentar el rendimiento
- Además:
    - En cada iteración del `while` se calcula el cuadrado de todos los $z$ 
    - Sin embargo, lo más correcto sería calcular los cuadrados solo para los $z$ que cumplieron con la condición $|z| < 4$ la iteración anterior
    - ¿Cuánto *speed-up* se obtiene si se implementa esta observación?